In [19]:
!pip -q install transformers faiss-cpu ranx tqdm ir_datasets

import os, math, time, json
from pathlib import Path
import numpy as np
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModel
import faiss
import ir_datasets
from ranx import Qrels, Run, evaluate

# ---------- Config ----------
WORKDIR = Path("/kaggle/working")
EMB_DIR = WORKDIR / "miracl_ar_contriever_emb_chunks"
RUN_DIR = WORKDIR / "runs"
EMB_DIR.mkdir(parents=True, exist_ok=True)
RUN_DIR.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "facebook/contriever"
DATASET = "miracl/ar/dev"     # queries + qrels
CORPUS  = "miracl/ar"         # corpus

CHUNK_SIZE = 50_000
BATCH_SIZE = 128
MAX_LENGTH = 256

TOPK = 1000

# IVF params (good for 2M docs)
N_LIST = 8192
N_PROBE = 16
TRAIN_VECS = 400_000  # better than 200k if possible

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------- Model ----------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

def mean_pooling(last_hidden_state, attention_mask):
    mask = attention_mask.unsqueeze(-1).type_as(last_hidden_state)
    summed = (last_hidden_state * mask).sum(dim=1)
    counts = mask.sum(dim=1).clamp(min=1e-9)
    return summed / counts

@torch.no_grad()
def encode_texts(texts, batch_size=BATCH_SIZE):
    embs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        tok = tokenizer(batch, padding=True, truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
        tok = {k: v.to(device) for k, v in tok.items()}
        with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
            out = model(**tok)
            emb = mean_pooling(out.last_hidden_state, tok["attention_mask"])
            emb = torch.nn.functional.normalize(emb, p=2, dim=1)
        embs.append(emb.detach().cpu().numpy().astype("float32"))
    return np.vstack(embs)

def chunk_paths(chunk_idx: int):
    emb_path = EMB_DIR / f"emb_{chunk_idx:05d}.npy"
    ids_path = EMB_DIR / f"ids_{chunk_idx:05d}.npy"
    meta_path = EMB_DIR / f"meta_{chunk_idx:05d}.json"
    return emb_path, ids_path, meta_path

# ---------- Load datasets ----------
ds = ir_datasets.load(DATASET)     # has queries + qrels + (docs_iter too)
corpus_ds = ir_datasets.load(CORPUS)

num_docs = corpus_ds.docs_count()
num_chunks = math.ceil(num_docs / CHUNK_SIZE)
print("Corpus docs:", num_docs, "Num chunks:", num_chunks)

# ---------- 1) Encode corpus in chunks (SKIP if exists) ----------
docs_iter = corpus_ds.docs_iter()
chunk_idx = 0
buf_ids, buf_texts = [], []
seen = 0

for d in tqdm(docs_iter, total=num_docs, desc="Streaming corpus"):
    doc_id = str(d.doc_id)
    title = (d.title or "").strip()
    text = (d.text or "").strip()
    full = (title + "\n" + text).strip() if title else text

    buf_ids.append(doc_id)
    buf_texts.append(full)
    seen += 1

    if len(buf_ids) == CHUNK_SIZE or seen == num_docs:
        emb_path, ids_path, meta_path = chunk_paths(chunk_idx)

        if emb_path.exists() and ids_path.exists():
            # skip
            pass
        else:
            t0 = time.time()
            emb = encode_texts(buf_texts)
            dt = time.time() - t0
            np.save(emb_path, emb)
            np.save(ids_path, np.array(buf_ids, dtype=object))
            meta_path.write_text(json.dumps({
                "chunk_idx": chunk_idx, "n": len(buf_ids), "seconds": dt
            }, ensure_ascii=False, indent=2), encoding="utf-8")
            print(f"✅ saved chunk {chunk_idx} n={len(buf_ids)} time={dt/60:.1f}m")

        buf_ids, buf_texts = [], []
        chunk_idx += 1

print("✅ All chunks ready.")

# ---------- 2) Build FAISS IVF index ----------
first = np.load(chunk_paths(0)[0])
dim = first.shape[1]
print("Embedding dim:", dim)

# collect training vectors
rng = np.random.default_rng(42)
train_buf = []
need = TRAIN_VECS

for ci in range(num_chunks):
    emb = np.load(chunk_paths(ci)[0]).astype("float32")
    if emb.shape[0] <= need:
        train_buf.append(emb)
        need -= emb.shape[0]
    else:
        idx = rng.choice(emb.shape[0], size=need, replace=False)
        train_buf.append(emb[idx])
        need = 0
    if need == 0:
        break

train_x = np.vstack(train_buf)
print("Train sample:", train_x.shape)

quantizer = faiss.IndexFlatIP(dim)
index = faiss.IndexIVFFlat(quantizer, dim, N_LIST, faiss.METRIC_INNER_PRODUCT)
print("Training IVF...")
index.train(train_x)
index.nprobe = N_PROBE
print("✅ IVF trained. Adding vectors...")

all_doc_ids = []
total = 0
for ci in range(num_chunks):
    emb = np.load(chunk_paths(ci)[0]).astype("float32")
    ids = np.load(chunk_paths(ci)[1], allow_pickle=True)
    index.add(emb)
    all_doc_ids.append(ids)
    total += emb.shape[0]
    if ci % 10 == 0:
        print(f"  added {ci}/{num_chunks-1}, total={total}")

all_doc_ids = np.concatenate(all_doc_ids)
print("Index total:", index.ntotal, "Docids:", len(all_doc_ids))

INDEX_PATH = WORKDIR / "faiss_miracl_ar_contriever_ivf.index"
DOCIDS_PATH = WORKDIR / "faiss_miracl_ar_docids.npy"
faiss.write_index(index, str(INDEX_PATH))
np.save(DOCIDS_PATH, all_doc_ids)
print("✅ Saved index + docids")

# ---------- 3) Encode queries ----------
queries = list(ds.queries_iter())
query_ids = [str(q.query_id) for q in queries]
query_texts = [q.text for q in queries]
q_emb = encode_texts(query_texts, batch_size=64)
print("Queries:", len(query_ids), "q_emb:", q_emb.shape)

# ---------- 4) Search + write run ----------
scores, idxs = index.search(q_emb.astype("float32"), TOPK)
run_path = RUN_DIR / "contriever_zero_miracl_ar_dev.run"
with open(run_path, "w", encoding="utf-8") as f:
    for i, qid in enumerate(query_ids):
        for rank, (doc_idx, score) in enumerate(zip(idxs[i], scores[i]), start=1):
            if doc_idx < 0:
                continue
            docid = str(all_doc_ids[int(doc_idx)])
            f.write(f"{qid} Q0 {docid} {rank} {float(score)} contriever_zero\n")
print("✅ Wrote run:", run_path)

# ---------- 5) Eval (qrels from SAME ds) ----------
qrels_dict = {}
for r in ds.qrels_iter():
    qrels_dict.setdefault(str(r.query_id), {})
    qrels_dict[str(r.query_id)][str(r.doc_id)] = int(r.relevance)

qrels = Qrels(qrels_dict)
run = Run.from_file(str(run_path), kind="trec")

metrics = ["map", "mrr", "ndcg@10", "recall@100"]
results = evaluate(qrels, run, metrics)
print(results)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Device: cuda
Corpus docs: 2061414 Num chunks: 42


Streaming corpus:   2%|▏         | 45544/2061414 [00:00<00:30, 65106.60it/s]/tmp/ipykernel_55/4293415551.py:56: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
Streaming corpus:   3%|▎         | 64452/2061414 [02:56<2:14:19, 247.77it/s]

✅ saved chunk 0 n=50000 time=2.9m


Streaming corpus:   6%|▌         | 114911/2061414 [05:51<2:05:15, 259.01it/s]

...

Streaming corpus: 100%|█████████▉| 2059790/2061414 [1:56:14<00:08, 196.22it/s]

✅ saved chunk 40 n=50000 time=2.8m


Streaming corpus: 100%|██████████| 2061414/2061414 [1:56:52<00:00, 293.97it/s]

✅ saved chunk 41 n=11414 time=0.6m
✅ All chunks ready.


Embedding dim: 768
Train sample: (400000, 768)
Training IVF...
✅ IVF trained. Adding vectors...
  added 0/41, total=50000
  added 10/41, total=550000
  added 20/41, total=1050000
  added 30/41, total=1550000
  added 40/41, total=2050000
Index total: 2061414 Docids: 2061414
✅ Saved index + docids
Queries: 2896 q_emb: (2896, 768)
✅ Wrote run: /kaggle/working/runs/contriever_zero_miracl_ar_dev.run
{'map': np.float64(0.0006234993661948514), 'mrr': np.float64(0.0006794800701671288), 'ndcg@10': np.float64(0.0007499007726032625), 'recall@100': np.float64(0.0024171270718232043)}
